## Ungraded Lab: Lambda Layer

This lab will show how you can define custom layers with the [Lambda](https://keras.io/api/layers/core_layers/lambda/) layer. You can either use [lambda functions](https://www.w3schools.com/python/python_lambda.asp) within the Lambda layer or define a custom function that the Lambda layer will call. Let's get started!

## Imports

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import backend as K

## Prepare the Dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Build the Model

Here, we'll use a Lambda layer to define a custom layer in our network. We're using a lambda function to get the absolute value of the layer input.

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Lambda(lambda x: tf.abs(x)), 
  tf.keras.layers.Dense(10, activation='softmax')
])

In [4]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 76us/sample - loss: 0.2272 - accuracy: 0.9349
Epoch 2/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0916 - accuracy: 0.9724
Epoch 3/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0641 - accuracy: 0.9801
Epoch 4/5
60000/60000 [==============================] - 4s 72us/sample - loss: 0.0478 - accuracy: 0.9848
Epoch 5/5
10000/10000 [==============================] - 0s 37us/sample - loss: 0.0745 - accuracy: 0.9773


[0.07451150287849014, 0.9773]

Another way to use the Lambda layer is to pass in a function defined outside the model. The code below shows how a custom ReLU function is used as a custom layer in the model.

## Relu

f(x)=\begin{cases}x&{\text{if }}x>0,\\0.01x&{\text{otherwise}}.\end{cases}

## Elu

y = ELU(x) = exp(x) − 1 ; if x<0

y = ELU(x) = x ; if x≥0

In [27]:
import math
def leaky_relu(x):
    cond = x > 0
    return tf.where(cond,x,0.01 * x)

def elu(x):
    cond = x > 0
    return tf.where(cond, x, 0.01 * (tf.math.exp(x) - 1 ))

def selu(x):
    lamb = 1.0507009873554804934193349852946
    alph = 1.6732632423543772848170429916717
    cond = x > 0
    return tf.where(cond, lamb * x, alph * (tf.math.exp(x) - alph))

def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / math.pi) * (x + 0.044715 * tf.pow(x, 3))))

def gelu(x):
    term = 1 + tf.math.tanh(tf.math.sqrt(2 / math.pi) * (𝑥 + 0.044715 * tf.math.pow(x,3)))
    return 0.5 * x * (term)

In [26]:
def my_relu(x):
    return K.maximum(-0.1, x)

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Lambda(gelu), 
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs=5)
model.evaluate(x_test, y_test)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 79us/sample - loss: 0.2532 - accuracy: 0.9289
Epoch 2/5
60000/60000 [==============================] - 5s 76us/sample - loss: 0.1093 - accuracy: 0.9676
Epoch 3/5
60000/60000 [==============================] - 5s 77us/sample - loss: 0.0728 - accuracy: 0.9773
Epoch 4/5
60000/60000 [==============================] - 5s 75us/sample - loss: 0.0538 - accuracy: 0.9830
Epoch 5/5
10000/10000 [==============================] - 0s 28us/sample - loss: 0.0839 - accuracy: 0.9736


[0.08394910928448662, 0.9736]

## gelu and elu perform better than relu, But selu performs worse. maybe work better in larger networks